In [1]:
import numpy as np
import seaborn as sns

In [2]:
from utils import load_citation

In [3]:
import networkx as nx

In [4]:
import scipy

In [5]:
adj, features, labels , idx_tr, idx_va, idx_te = load_citation("cora", "", False)

In [6]:
X = features.numpy()

In [7]:
y = labels.numpy()

In [8]:
X_tr = X[idx_tr]

In [9]:
X_tr.shape

(270, 1433)

In [10]:
X_tr[0,75]

0.0

## Make scatter matrices
Between classes scatter matrix measures the distance between each class centers and data center.
$$S_B = \sum_{c} N_c (\mu_c - \bar{x})(\mu_c - \bar{x})^\top$$
Within classes scatter matrix measures the distance between data of a class and that class' center.
$$S_W = \sum_{c} \sum_{i \in c} (x_i - \mu_c)(x_i - \mu_c)^\top$$

In [11]:
y_tr = y[idx_tr]

In [12]:
X_tr_bar = np.mean(X_tr, axis=0)

In [13]:
S_B = np.zeros((1433, 1433))
S_W = np.zeros((1433, 1433))

In [15]:
for c in range(6):
    class_c_idx = np.where(y_tr == c)[0]
    N_c = len(class_c_idx)
    mu_c = np.mean(X_tr[class_c_idx], axis=0)
    S_B += N_c * (mu_c - X_tr_bar).reshape(1433, 1) @ (mu_c - X_tr_bar).reshape(1, 1433) 

In [16]:
for c in range(6):
    class_c_idx = np.where(y_tr == c)[0]
    mu_c = np.mean(X_tr[class_c_idx], axis=0)
    for i in class_c_idx:
        S_W += (X_tr[i] - mu_c).reshape(1433, 1) @ (X_tr[i] - mu_c).reshape(1, 1433) 

In [17]:
l, v = np.linalg.eigh(S_B)

In [18]:
l[l < 0] = 0  # S_B is not exactly symmetric due to numerical errors

In [19]:
S_B_p_half = v @ np.diag(l**0.5) @ v.T

In [21]:
S_W_inv = np.linalg.inv(S_W + 0.01 * np.eye(1433))

In [22]:
T = S_B_p_half @ S_W_inv @ S_B_p_half

In [23]:
lT, vT = np.linalg.eigh(T)

In [47]:
g = nx.from_numpy_array(adj.int().to_dense().numpy(), create_using=nx.Graph())

In [48]:
g.number_of_edges(), g.number_of_nodes()

(4676, 3327)

In [56]:
for i in nx.connected_components(g):
    print(len(i))
    break

2


In [49]:
L = nx.laplacian_matrix(g)

In [50]:
L = L.todense()

In [51]:
lL, vL = np.linalg.eigh(L)

In [71]:
max(vL[:,1])

matrix([[0.11397922]])